<a href="https://colab.research.google.com/github/KiaroRB/data-analysis-projects/blob/main/bronze_data_ingestion_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
      .appName("bronze_ingestion")\
      .getOrCreate()

In [3]:
!pip install -q gdown

In [7]:
import gdown

file_id = "1LHeH732nph-c0_rrkMJlXuX0xKCqkKHz"
gdown.download(id=file_id, output="online_retail.csv", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1LHeH732nph-c0_rrkMJlXuX0xKCqkKHz
To: /content/online_retail.csv
100%|██████████| 45.6M/45.6M [00:01<00:00, 37.9MB/s]


'online_retail.csv'

In [16]:
df = spark.read.csv(
    "online_retail.csv",
    header=True,
    inferSchema=True,
    sep = ";"
)

In [17]:
df.write.mode("overwrite").parquet("bronze_online_retail_raw")

In [23]:
print("Total de registros:", df.count())
print("\nTotal de columnas:", len(df.columns))
print("\n REGISTRO DEL DATASET")
df.limit(5).show(truncate=False)
print("\n Verificación de datos")
df.printSchema()

Total de registros: 541909

Total de columnas: 8

 REGISTRO DEL DATASET
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |1/12/2010 8:26|2,55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |1/12/2010 8:26|3,39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |1/12/2010 8:26|2,75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |1/12/2010 8:26|3,39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |1/12/2010 8:26|3,39     |17850     |

In [ ]:
♦